## Imports

In [1]:
import os
import pandas as pd
import numpy as np
from importlib import reload
import src.utils as utils
import glob

In [2]:
chroms_names = ['MT', 'X'] + [str(x) for x in range(1,23)]

GENOMIC_DESCRIPTION_COL = 'Genomic Description (GRCh37)'

## Detour to Examine the Internal Kafeen State (Azaeiz Method)

In [3]:
df1 = pd.concat(dfs1)

NameError: name 'dfs1' is not defined

In [ ]:
df.head()

,FINAL_PATHOGENICITY,FINAL_PRED,CURATED_PATHOGENICITY,TOTAL_NUM_PREDS,NUM_PATH_PREDS
Genomic Description GRCh37,,,,,
X:9431334:T>C,Unknown significance,NaN,NaN,NaN,NaN
X:9431341:T>C,Unknown significance,NaN,NaN,NaN,NaN
X:9431342:G>GT,Unknown significance,NaN,NaN,NaN,NaN
X:9431375:G>A,Unknown significance,NaN,NaN,NaN,NaN
X:9431417:A>G,Unknown significance,NaN,NaN,NaN,NaN


In [ ]:
df['PREDICTION_RATIO'] = df1['NUM_PATH_PREDS'] / df1['TOTAL_NUM_PREDS']

In [4]:
df1['INTERNAL_STATE'] = 'VUS'
df1.loc[df1['TOTAL_NUM_PREDS'] >= 5, 'INTERNAL_STATE'] = 'D'
df1.loc[(df1['TOTAL_NUM_PREDS'] >= 5) & (df1['PREDICTION_RATIO'] < 0.6), 'INTERNAL_STATE'] = 'B'
df1

NameError: name 'df1' is not defined

In [182]:
df1.INTERNAL_STATE.value_counts()

INTERNAL_STATE
VUS    2552823
D        91470
B        48068
Name: count, dtype: int64

In [184]:
df1[['FINAL_PATHOGENICITY', 'INTERNAL_STATE']].value_counts()

FINAL_PATHOGENICITY   INTERNAL_STATE
Unknown significance  VUS               2307346
Benign                VUS                225772
Unknown significance  D                   81886
Likely benign         B                   30956
Unknown significance  B                   15243
Pathogenic            VUS                 10053
Likely benign         VUS                  7412
Pathogenic            D                    6186
Likely pathogenic     VUS                  2217
                      D                    2166
Benign                B                    1053
                      D                     736
Pathogenic            B                     450
Likely benign         D                     449
Likely pathogenic     B                     344
Benign*               D                      47
                      VUS                    23
                      B                      22
Name: count, dtype: int64

# Data Selection

### Constructing df

In [5]:
chromosome_dir = 'split_vcf_chromosomes_csvs'

if not os.path.isdir(chromosome_dir) or len(os.listdir(chromosome_dir)) == 0:
    if not os.path.isdir(chromosome_dir):
        os.makedirs(chromosome_dir)

    compressed_dir = 'split_vcf_chromosomes_csvs_compressed'
    for x in os.scandir(compressed_dir):
        utils.extract_tarball(x)


In [6]:
compressed_dir = 'compressed_chromosome_w_ddg'
if not os.path.isdir(compressed_dir):
    os.mkdir(compressed_dir)

for x in os.scandir(compressed_dir):
    utils.extract_tarball(x)

Decompression complete. Decompressed content saved to 'compressed_chromosome_w_ddg/chromosome_18_w_ddg.tar'
Unarchiving complete
Removed archive file compressed_chromosome_w_ddg/chromosome_18_w_ddg.tar
Decompression complete. Decompressed content saved to 'compressed_chromosome_w_ddg/chromosome_19_w_ddg.tar'
Unarchiving complete
Removed archive file compressed_chromosome_w_ddg/chromosome_19_w_ddg.tar
Decompression complete. Decompressed content saved to 'compressed_chromosome_w_ddg/chromosome_X_w_ddg.tar'
Unarchiving complete
Removed archive file compressed_chromosome_w_ddg/chromosome_X_w_ddg.tar
Decompression complete. Decompressed content saved to 'compressed_chromosome_w_ddg/chromosome_22_w_ddg.tar'
Unarchiving complete
Removed archive file compressed_chromosome_w_ddg/chromosome_22_w_ddg.tar
Decompression complete. Decompressed content saved to 'compressed_chromosome_w_ddg/chromosome_13_w_ddg.tar'
Unarchiving complete
Removed archive file compressed_chromosome_w_ddg/chromosome_13_w_

In [7]:
dfs = []
id_cols = ['CHROM', 'POS', 'REF', 'ALT']
dup_cols = [f"{base_col}_{alt}" for base_col in id_cols for alt in ['x', 'y']]
for f in os.scandir('chromosomes_w_ddg'):
    df = pd.read_csv(f.path, index_col=0, encoding='utf-8', low_memory=False)
    df = df.drop(columns=dup_cols)
    dfs.append(df)
df = pd.concat(dfs).map(utils.safe_decode)
df

/var/folders/3f/85zthwnn17dfrdm_swynn0qr0000gn/T/ipykernel_70840/827858591.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs).map(utils.safe_decode)


,TYPE,GENE,gnomad_MAX_MAF,gnomad_MAX_MAF_SOURCE,gnomad_AC_afr_pass,gnomad_AC_amr_pass,gnomad_AC_eas_pass,gnomad_AC_nfe_pass,gnomad_AN_afr_pass,gnomad_AN_amr_pass,...,Max MAF Source,Surface Area,Normalized SA,Confidence Score,ddG,|ddG|,ca...,in...,calc...,second part
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686358:T>C,SNV,PAX1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,172.951510,0.752780,32.73,-1.4,1.4,NaN,NaN,NaN,NaN
20:21686361:C>T,SNV,PAX1,0.000067,gnomad_AF_nfe_pass,0.0,0.0,0.0,1.0,8602.0,810.0,...,gnomad_AF_nfe_pass,139.065932,0.883870,38.06,-0.0,0.0,NaN,NaN,NaN,NaN
20:21686379:G>A,SNV,PAX1,0.000105,gnomad_AF_nfe_pass,0.0,0.0,0.0,1.0,1492.0,274.0,...,gnomad_AF_nfe_pass,180.703642,0.670700,37.23,-0.3,0.3,NaN,NaN,NaN,NaN
20:21686382:C>T,SNV,PAX1,0.031875,gnomad_AF_afr_pass,319.0,1.0,0.0,4.0,10008.0,1046.0,...,gnomad_AF_afr_pass,76.425233,0.601022,34.61,-0.3,0.3,NaN,NaN,NaN,NaN
20:21686390:G>T,SNV,PAX1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,99.401913,0.578281,31.28,-0.6,0.6,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115915:C>T,SNV,GATA3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,136.946574,0.870607,47.29,-0.3,0.3,NaN,NaN,NaN,NaN
10:8115916:C>G,SNV,GATA3,0.000065,gnomad_AF_nfe_pass,0.0,0.0,0.0,1.0,8702.0,848.0,...,gnomad_AF_nfe_pass,136.946574,0.870607,47.29,-0.3,0.3,NaN,NaN,NaN,NaN
10:8115916:C>T,SNV,GATA3,0.000123,gnomad_AF_afr_pass,2.0,1.0,0.0,2.0,16248.0,34588.0,...,gnomad_AF_afr_pass,136.946574,0.870607,47.29,-0.2,0.2,NaN,NaN,NaN,NaN


### Columns to Drop

In [8]:
cols_to_drop = []

# Colummns containing various identifiers from other sources
id_cols = df.columns[df.columns.str.contains('ID') & ~df.columns.str.contains('CONFIDENCE')].tolist()

# gnomad columns - mostly just allele frequency filters
gnomad_filter_cols = df.columns[df.columns.str.contains('gnomad') & df.columns.str.contains('pass')].tolist()
gnomad_filter_cols.append('gnomad_filters')

# Columns containing minor allele frequency (MAF)
# max_maf_cols = df.columns[df.columns.str.contains('MAF')].tolist()

# Errant columns, unknown origin
errant_cols = [' second part', ' in...', ' ca...', ' calc...']

# DVD/Kafeen Columns
dvd_kafeen_cols = ['CLASSIFY_TRAIL', 'CLASSIFY_TRAIL_KEYS', 'FINAL_COMMENTS', 'FINAL_PATHOGENICITY_REASON', 'CURATED_DECISION_DATE', 'PREF_TRANSCRIPT']

# Columns containing variant location information
genomic_protein_loc_cols = ['VEP_HGVS_P', 'VEP_HGVS_C', 'CLINVAR_HGVS_P', 'CLINVAR_HGVS_C', 'Protein Position']
gene_description_cols = ['HGMD_OMIM_REF'] + ['VEP_FEATURE'] # Unsure what VEP_FEATURE is, but it looks like an identifier

# Comment columns
comment_cols = ['FINAL_COMMENTS', 'CURATED_COMMENTS']

# Misc columns
misc = ['VEP_OTHER_FEATURES']

cols_to_drop += id_cols + gnomad_filter_cols + errant_cols + dvd_kafeen_cols + genomic_protein_loc_cols + gene_description_cols + misc + comment_cols

In [9]:
keep_cols = ['NUM_PATH_PREDS',
 'TOTAL_NUM_PREDS',
 'FINAL_PRED',
 'GENE',
 'OVERALL_MAX_MAF',
 'OVERALL_MAX_MAF_SOURCE',
 'FINAL_PATHOGENICITY',
 'CURATED_PATHOGENICITY']


### Alternate Approach

In [10]:
feature_sorting_df = pd.read_csv('feature_sorting_updated.csv', header=None, index_col=0)

In [11]:
feature_sorting_df.loc['Definitely Keep (used previously)'].dropna().tolist()

['NUM_PATH_PREDS',
 'TOTAL_NUM_PREDS',
 'FINAL_PRED',
 'GENE',
 'OVERALL_MAX_MAF',
 'OVERALL_MAX_MAF_SOURCE',
 'FINAL_PATHOGENICITY',
 'CURATED_PATHOGENICITY']

In [12]:
kept_features_df = feature_sorting_df.loc[~feature_sorting_df.index.str.contains('Not Keep'), :]

In [13]:
kept_features_list = sorted(kept_features_df.unstack().dropna().unique().tolist())
len(kept_features_list)

46

In [14]:
pathogenicity_label_cols = ['CLINVAR_CLNSIG', 'CLINVAR_PATHOGENIC', 'CURATED_PATHOGENICITY']

## Selecting Columns

In [15]:
kept_features_list

['CADD Phred',
 'CLINVAR_CLNSIG',
 'CLINVAR_PATHOGENIC',
 'CURATED_PATHOGENICITY',
 'Confidence Score',
 'Consequence',
 'DBNSFP_GERP_RS',
 'DBNSFP_GERP_RS_PRED',
 'DBNSFP_INTERPRO_DOMAIN',
 'DBNSFP_LRT_PRED',
 'DBNSFP_LRT_SCORE',
 'DBNSFP_MUTATIONTASTER_PRED',
 'DBNSFP_MUTATIONTASTER_SCORE',
 'DBNSFP_PHYLOP30WAY_MAMMALIAN',
 'DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED',
 'DBNSFP_POLYPHEN2_HDIV_PRED',
 'DBNSFP_POLYPHEN2_HDIV_SCORE',
 'DBNSFP_SIFT_PRED',
 'DBNSFP_SIFT_SCORE',
 'FINAL_PATHOGENICITY',
 'FINAL_PRED',
 'GENE',
 'Impact',
 'Interpro Domain',
 'Max MAF (%)',
 'Max MAF Source',
 'NUM_PATH_PREDS',
 'Normalized SA',
 'OVERALL_MAX_MAF',
 'OVERALL_MAX_MAF_SOURCE',
 'Surface Area',
 'TOTAL_NUM_PREDS',
 'TYPE',
 'UNIPROT_DOMAIN',
 'VEP_CADD_PHRED',
 'VEP_CADD_RAW',
 'VEP_CONSEQUENCE',
 'VEP_FEATURE',
 'VEP_IMPACT',
 'VEP_POLYPHEN_PRED',
 'VEP_POLYPHEN_SCORE',
 'VEP_PROTEIN_POS',
 'VEP_SIFT_SCORE',
 'Variant Classification',
 'ddG',
 '|ddG|']

In [16]:
working_df = df[kept_features_list]
working_df

,CADD Phred,CLINVAR_CLNSIG,CLINVAR_PATHOGENIC,CURATED_PATHOGENICITY,Confidence Score,Consequence,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_INTERPRO_DOMAIN,DBNSFP_LRT_PRED,...,VEP_CONSEQUENCE,VEP_FEATURE,VEP_IMPACT,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_PROTEIN_POS,VEP_SIFT_SCORE,Variant Classification,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686358:T>C,23.60,NaN,NaN,NaN,32.73,missense_variant,4.28,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.596,3,0.00,Unknown significance,-1.4,1.4
20:21686361:C>T,23.00,NaN,NaN,NaN,38.06,missense_variant,4.28,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.917,4,0.00,Unknown significance,-0.0,0.0
20:21686379:G>A,22.90,NaN,NaN,NaN,37.23,missense_variant,2.32,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.787,10,0.00,Unknown significance,-0.3,0.3
20:21686382:C>T,21.80,NaN,NaN,NaN,34.61,missense_variant,2.11,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.804,11,0.01,Benign,-0.3,0.3
20:21686390:G>T,22.10,NaN,NaN,NaN,31.28,missense_variant,3.51,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,B,0.005,14,0.04,Unknown significance,-0.6,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115915:C>T,27.20,NaN,NaN,NaN,47.29,missense_variant,5.59,C,.&.&.,U,...,missense_variant,NM_001002295.2,MODERATE,D,1.000,422,0.01,Likely pathogenic,-0.3,0.3
10:8115916:C>G,27.50,NaN,NaN,NaN,47.29,missense_variant,5.59,C,.&.&.,U,...,missense_variant,NM_001002295.2,MODERATE,D,1.000,422,0.00,Unknown significance,-0.3,0.3
10:8115916:C>T,29.20,NaN,NaN,NaN,47.29,missense_variant,5.59,C,.&.&.,U,...,missense_variant,NM_001002295.2,MODERATE,D,1.000,422,0.00,Unknown significance,-0.2,0.2


#### Exploring Prediction Columns

In [17]:
# In GERP, RS > 0 => C == conserved
# In GERP, RS < 0 => N == not conserved (?)
gerp_pos_df = df.loc[df.DBNSFP_GERP_RS > 0, ['DBNSFP_GERP_RS_PRED', 'DBNSFP_GERP_RS']]
gerp_pos_df.head()

,DBNSFP_GERP_RS_PRED,DBNSFP_GERP_RS
Genomic Description (GRCh37),,
20:21686358:T>C,C,4.28
20:21686361:C>T,C,4.28
20:21686379:G>A,C,2.32
20:21686382:C>T,C,2.11
20:21686390:G>T,C,3.51


In [18]:
gerp_neg_df = df.loc[df.DBNSFP_GERP_RS < 0, ['DBNSFP_GERP_RS_PRED', 'DBNSFP_GERP_RS']]
gerp_neg_df.head()

,DBNSFP_GERP_RS_PRED,DBNSFP_GERP_RS
Genomic Description (GRCh37),,
20:21686402:G>C,N,-0.290
20:21686457:G>A,N,-0.296
20:21686459:G>A,N,-8.940
20:21686468:G>A,N,-1.940
20:21686502:G>T,N,-0.914


In [19]:
sift_df = df.loc[:, df.columns.str.contains('SIFT')]
sift_df.head()

,VEP_SIFT_SCORE,VEP_SIFT_PRED,DBNSFP_SIFT_PRED,DBNSFP_SIFT_SCORE
Genomic Description (GRCh37),,,,
20:21686358:T>C,0.00,D-LC,D&.,0.001&.
20:21686361:C>T,0.00,D-LC,D&.,0.0&.
20:21686379:G>A,0.00,D-LC,D&.,0.0&.
20:21686382:C>T,0.01,D-LC,D&.,0.002&.
20:21686390:G>T,0.04,D-LC,D&.,0.006&.


In [20]:
df.loc[~df.VEP_SIFT_PRED.isna(), df.columns.str.contains('SIFT')]

,VEP_SIFT_SCORE,VEP_SIFT_PRED,DBNSFP_SIFT_PRED,DBNSFP_SIFT_SCORE
Genomic Description (GRCh37),,,,
20:21686358:T>C,0.00,D-LC,D&.,0.001&.
20:21686361:C>T,0.00,D-LC,D&.,0.0&.
20:21686379:G>A,0.00,D-LC,D&.,0.0&.
20:21686382:C>T,0.01,D-LC,D&.,0.002&.
20:21686390:G>T,0.04,D-LC,D&.,0.006&.
...,...,...,...,...
10:8115915:C>T,0.01,D,D&.&D,0.014&.&0.014
10:8115916:C>G,0.00,D,D&.&D,0.008&.&0.008
10:8115916:C>T,0.00,D,D&.&D,0.002&.&0.003


In [21]:
##INFO=<ID=DBNSFP_POLYPHEN2_HDIV_PRED,Number=1,Type=String,Description="\"(from dbNSFP) Polyphen2 prediction based on HumDiv, D (probably damaging, HDIV score in [0.957,1] or rankscore in [0.55859,0.91137]), P (possibly damaging, HDIV score in [0.454,0.956] or rankscore in [0.37043,0.55681]) and B (benign, HDIV score in [0,0.452] or rankscore in [0.03061,0.36974]). Score cutoff for binary classification is 0.5 for HDIV score or 0.38028 for rankscore, i.e. the prediction is neutral if the HDIV score is smaller than 0.5 (rankscore is smaller than 0.38028), and deleterious if the HDIV score is larger than 0.5 (rankscore is larger than 0.38028). Multiple entries are separated by ;, corresponding to Uniprot_acc.\"">
##INFO=<ID=DBNSFP_POLYPHEN2_HDIV_SCORE,Number=1,Type=String,Description="\"(from dbNSFP) Polyphen2 score based on HumDiv, i.e. hdiv_prob. The score ranges from 0 to 1. Multiple entries separated by ;, corresponding to Uniprot_acc.\"">
polyphen_df = df.loc[:, df.columns.str.contains('POLYPHEN')]
polyphen_df.head()

,VEP_POLYPHEN_SCORE,VEP_POLYPHEN_PRED,DBNSFP_POLYPHEN2_HDIV_PRED,DBNSFP_POLYPHEN2_HDIV_SCORE
Genomic Description (GRCh37),,,,
20:21686358:T>C,0.596,P,P&.,0.596&.
20:21686361:C>T,0.917,P,P&.,0.917&.
20:21686379:G>A,0.787,P,P&.,0.787&.
20:21686382:C>T,0.804,P,P&.,0.804&.
20:21686390:G>T,0.005,B,B&.,0.005&.


In [22]:
polyphen_df.VEP_POLYPHEN_PRED.value_counts()

VEP_POLYPHEN_PRED
D    55263
B    51063
P    22231
U      210
Name: count, dtype: int64

In [23]:
##INFO=<ID=DBNSFP_MUTATIONTASTER_PRED,Number=1,Type=String,Description="\"(from dbNSFP) MutationTaster prediction, A (disease_causing_automatic), D (disease_causing), N (polymorphism) or P (polymorphism_automatic). The score cutoff between D and N is 0.5 for MTnew and 0.31733 for the rankscore.\"">
##INFO=<ID=DBNSFP_MUTATIONTASTER_SCORE,Number=1,Type=String,Description="\"(from dbNSFP) MutationTaster p-value (MTori), ranges from 0 to 1. Multiple scores are separated by ;. Information on corresponding transcript(s) can be found by querying http://www.mutationtaster.org/ChrPos.html\"">
mutationtaster_df = df.loc[:, df.columns.str.contains('MUTATIONTASTER')]
mutationtaster_df.head()

,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE
Genomic Description (GRCh37),,
20:21686358:T>C,N,0.506194
20:21686361:C>T,N,0.758794
20:21686379:G>A,N,0.999272
20:21686382:C>T,N,0.998176
20:21686390:G>T,N,0.999492


In [24]:
for i, v in mutationtaster_df.DBNSFP_MUTATIONTASTER_PRED.value_counts().items():
    print(i,v)

D&D 21946
D 17739
D&D&D 15324
N&N 12588
N 10410
D&D&D&D 9660
N&N&N 6795
D&D&D&D&D 5238
D&D&D&D&D&D 3929
N&N&N&N 2700
N&N&N&N&N 2023
D&D&D&D&D&D&D&D 1904
D&D&D&D&D&D&D 1637
D&N 1555
D&D&D&D&D&D&D&D&D 1550
N&N&N&N&N&N 1368
D&D&D&D&D&D&D&D&D&D&D 1123
N&N&N&N&N&N&N&N 1033
D&N&N 961
N&N&N&N&N&N&N 838
D&D&N 822
D&D&D&D&D&D&D&D&D&D&D&D&D&D&D&D 573
N&N&N&N&N&N&N&N&N 541
D&N&N&N 337
D&D&D&N 334
D&D&D&D&D&D&D&D&D&D&D&D&D&D&D 329
A&A 318
D&D&D&D&D&D&D&D&D&D 275
D&D&D&D&N 229
D&N&N&N&N 164
P&P 155
D&D&N&N 148
D&D&D&D&N&N&N&N 138
D&D&D&D&D&D&D&D&N 129
D&D&D&N&N 128
D&D&D&D&N&N 124
D&D&D&D&D&D&N&N&N 123
D&N&N&N&N&N&N 119
D&D&N&N&N 119
D&D&D&D&D&D&N 96
D&D&D&D&D&D&D&D&D&D&D&D 94
P 89
A 88
N&N&N&N&N&N&N&N&N&N&N&N 84
D&D&D&D&D&N 84
D&D&D&D&D&D&D&N&N 83
D&D&D&D&D&D&D&D&D&D&D&D&D&D&D&N 76
A&A&A&A 70
D&D&D&D&N&N&N&N&N 69
D&D&D&D&D&D&D&D&D&D&N&N 69
P&P&P 68
D&D&N&N&N&N 62
D&D&D&N&N&N 58
D&N&N&N&N&N&N&N&N 54
D&D&D&N&N&D 47
D&D&N&N&N&N&N 46
A&A&A 44
D&D&D&D&D&D&N&N 44
A&A&A&A&A&A&A&A&A&A&A 42
D&D&D&D&D&D&D&D

In [25]:
##INFO=<ID=DBNSFP_LRT_PRED,Number=1,Type=String,Description="\"(from dbNSFP) LRT prediction, D(eleterious), N(eutral) or U(nknown), which is not solely determined by the score.\"">
##INFO=<ID=DBNSFP_LRT_SCORE,Number=1,Type=String,Description="\"(from dbNSFP) The original LRT two-sided p-value (LRTori), ranges from 0 to 1.\"">
lrt_df = df.loc[:, df.columns.str.contains('LRT')]
lrt_df.head()

,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE
Genomic Description (GRCh37),,
20:21686358:T>C,NaN,NaN
20:21686361:C>T,NaN,NaN
20:21686379:G>A,NaN,NaN
20:21686382:C>T,NaN,NaN
20:21686390:G>T,NaN,NaN


In [26]:
##INFO=<ID=DBNSFP_PHYLOP30WAY_MAMMALIAN,Number=1,Type=String,Description="\"(from dbNSFP) phyloP (phylogenetic p-values) conservation score based on the multiple alignments of 30 mammalian genomes (including human). The larger the score, the more conserved the site. Scores range from -20 to 1.312 in dbNSFP.\"">
##INFO=<ID=DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED,Number=.,Type=String,Description="NA">
phylop_df = df.loc[:, df.columns.str.contains('PHYLOP')]
phylop_df.head()

,DBNSFP_PHYLOP30WAY_MAMMALIAN,DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED
Genomic Description (GRCh37),,
20:21686358:T>C,1.111,C
20:21686361:C>T,0.118,N
20:21686379:G>A,1.147,C
20:21686382:C>T,0.106,N
20:21686390:G>T,1.147,C


In [27]:
df.columns[df.columns.str.contains('PHYLOP')]

Index(['DBNSFP_PHYLOP30WAY_MAMMALIAN', 'DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED'], dtype='object')

In [28]:
score_cols = [
    'VEP_SIFT_SCORE', 
    'VEP_POLYPHEN_SCORE',
    'DBNSFP_SIFT_SCORE',
    'DBNSFP_POLYPHEN2_HDIV_SCORE',
    'DBNSFP_LRT_SCORE',
    'DBNSFP_MUTATIONTASTER_SCORE',
    'DBNSFP_GERP_RS',
    'DBNSFP_PHYLOP30WAY_MAMMALIAN'
]

predictor_cols = [
    'VEP_SIFT_PRED', 
    'VEP_POLYPHEN_PRED',
    'DBNSFP_SIFT_PRED',
    'DBNSFP_POLYPHEN2_HDIV_PRED',
    'DBNSFP_LRT_PRED',
    'DBNSFP_MUTATIONTASTER_PRED',
    'DBNSFP_GERP_RS_PRED',
    'DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED'
]

### Exploring Other Columns in working_df

In [29]:
domain_df = working_df.loc[:, working_df.columns.str.contains('Domain') | working_df.columns.str.contains('DOMAIN')]
domain_cols = domain_df.columns.tolist()
domain_df.head()

,DBNSFP_INTERPRO_DOMAIN,Interpro Domain,UNIPROT_DOMAIN
Genomic Description (GRCh37),,,
20:21686358:T>C,.&.,NaN,NaN
20:21686361:C>T,.&.,NaN,NaN
20:21686379:G>A,.&.,NaN,NaN
20:21686382:C>T,.&.,NaN,NaN
20:21686390:G>T,.&.,NaN,NaN


In [30]:
##INFO=<ID=VEP_CADD_RAW,Number=1,Type=String,Description="\"VEP annotated CADD raw score.\"">
##INFO=<ID=VEP_CADD_PHRED,Number=1,Type=String,Description="\"VEP annotated CADD phred scaled score.\"">
cadd_df = working_df.loc[:, working_df.columns.str.contains('Cadd') | working_df.columns.str.contains('CADD')]
cadd_cols = cadd_df.columns.tolist()
cadd_df.head()

,CADD Phred,VEP_CADD_PHRED,VEP_CADD_RAW
Genomic Description (GRCh37),,,
20:21686358:T>C,23.6,23.6,3.009015
20:21686361:C>T,23.0,23.0,2.751772
20:21686379:G>A,22.9,22.9,2.662314
20:21686382:C>T,21.8,21.8,2.257539
20:21686390:G>T,22.1,22.1,2.319001


In [31]:
# FINAL_PATHOGENICITY: "Final curated pathogenicity" -- includes the DVD manual override
# FINAL_PRED: "Final prediction consensus based on majority vote of prediction scores" -- should be if NUM_PATH_PREDS / TOTAL_NUM_PREDS >= 0.5 -> Likely pathogenic

# Our hypothesis - FINAL_PRED is actually the full Azaiez method result, not just the result of voting from the pathogenicty predictions
#                - FINAL_PATHOGENICTY is actually the post curation (overridden) label and is what is displayed on the dvd
dvd_labeling_cols = ['NUM_PATH_PREDS', 'TOTAL_NUM_PREDS', 'FINAL_PRED', 'FINAL_PATHOGENICITY']

In [32]:
maf_df = working_df.loc[:, ['OVERALL_MAX_MAF', 'OVERALL_MAX_MAF_SOURCE']]
maf_cols = maf_df.columns.tolist()
maf_df.head()

,OVERALL_MAX_MAF,OVERALL_MAX_MAF_SOURCE
Genomic Description (GRCh37),,
20:21686358:T>C,NaN,NaN
20:21686361:C>T,6.65956e-05,gnomad_AF_nfe_pass
20:21686379:G>A,0.000104932,gnomad_AF_nfe_pass
20:21686382:C>T,0.0318745,gnomad_AF_afr_pass
20:21686390:G>T,NaN,NaN


In [33]:
other_ddg_file_cols = ['Confidence Score', 'Max MAF (%)', 'Max MAF Source', 'Surface Area', 'Normalized SA', 'ddG', '|ddG|']
other_dvd_cols = ['VEP_IMPACT', 'VEP_CONSEQUENCE', 'VEP_PROTEIN_POS']
classification_cols = ['FINAL_PATHOGENICITY', 'FINAL_PRED', 'NUM_PATH_PREDS', 'TOTAL_NUM_PREDS']
misc_details_cols = ['GENE', 'TYPE']

### Identifying Duplicate Columns

In [34]:
working_df = df[kept_features_list]
working_df

,CADD Phred,CLINVAR_CLNSIG,CLINVAR_PATHOGENIC,CURATED_PATHOGENICITY,Confidence Score,Consequence,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_INTERPRO_DOMAIN,DBNSFP_LRT_PRED,...,VEP_CONSEQUENCE,VEP_FEATURE,VEP_IMPACT,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_PROTEIN_POS,VEP_SIFT_SCORE,Variant Classification,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686358:T>C,23.60,NaN,NaN,NaN,32.73,missense_variant,4.28,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.596,3,0.00,Unknown significance,-1.4,1.4
20:21686361:C>T,23.00,NaN,NaN,NaN,38.06,missense_variant,4.28,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.917,4,0.00,Unknown significance,-0.0,0.0
20:21686379:G>A,22.90,NaN,NaN,NaN,37.23,missense_variant,2.32,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.787,10,0.00,Unknown significance,-0.3,0.3
20:21686382:C>T,21.80,NaN,NaN,NaN,34.61,missense_variant,2.11,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,P,0.804,11,0.01,Benign,-0.3,0.3
20:21686390:G>T,22.10,NaN,NaN,NaN,31.28,missense_variant,3.51,C,.&.,NaN,...,missense_variant,NM_006192.5,MODERATE,B,0.005,14,0.04,Unknown significance,-0.6,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115915:C>T,27.20,NaN,NaN,NaN,47.29,missense_variant,5.59,C,.&.&.,U,...,missense_variant,NM_001002295.2,MODERATE,D,1.000,422,0.01,Likely pathogenic,-0.3,0.3
10:8115916:C>G,27.50,NaN,NaN,NaN,47.29,missense_variant,5.59,C,.&.&.,U,...,missense_variant,NM_001002295.2,MODERATE,D,1.000,422,0.00,Unknown significance,-0.3,0.3
10:8115916:C>T,29.20,NaN,NaN,NaN,47.29,missense_variant,5.59,C,.&.&.,U,...,missense_variant,NM_001002295.2,MODERATE,D,1.000,422,0.00,Unknown significance,-0.2,0.2


#### Consquence

In [35]:
working_df.loc[working_df.VEP_CONSEQUENCE != working_df.Consequence, ['VEP_CONSEQUENCE', 'Consequence']]

,VEP_CONSEQUENCE,Consequence
Genomic Description (GRCh37),,


In [36]:
working_df = working_df.drop(columns=['VEP_CONSEQUENCE'])

#### CADD

In [37]:
working_df.columns[working_df.columns.str.contains('CADD') | working_df.columns.str.contains('Cadd')]

Index(['CADD Phred', 'VEP_CADD_PHRED', 'VEP_CADD_RAW'], dtype='object')

These values are all missing even if it says they're not matching. Also are all MNV -> dropping these then re-examining

In [38]:
working_df.loc[working_df.VEP_CADD_PHRED != working_df['CADD Phred'], ['TYPE', 'CADD Phred', 'VEP_CADD_PHRED']]

,TYPE,CADD Phred,VEP_CADD_PHRED
Genomic Description (GRCh37),,,
20:61468569:CCT>GCC,MNV,NaN,NaN
2:228111435:TG>CA,MNV,NaN,NaN
2:73613119:GG>CA,MNV,NaN,NaN
2:73829349:GCTC>ACAT,MNV,NaN,NaN
18:44102214:GC>TT,MNV,NaN,NaN
18:44219603:AG>CC,MNV,NaN,NaN
4:6302568:TC>AG,MNV,NaN,NaN
4:6303899:CGC>GAG,MNV,NaN,NaN
4:6304085:TCA>CCG,MNV,NaN,NaN


In [39]:
working_df = working_df[working_df.VEP_CADD_PHRED == working_df['CADD Phred']]

In [40]:
working_df.loc[working_df.VEP_CADD_PHRED != working_df['CADD Phred'], ['TYPE', 'CADD Phred', 'VEP_CADD_PHRED']]

,TYPE,CADD Phred,VEP_CADD_PHRED
Genomic Description (GRCh37),,,


In [41]:
working_df = working_df.drop(columns='VEP_CADD_PHRED')

##### Impact

In [42]:
working_df[working_df['VEP_IMPACT'] != working_df['Impact']]

,CADD Phred,CLINVAR_CLNSIG,CLINVAR_PATHOGENIC,CURATED_PATHOGENICITY,Confidence Score,Consequence,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_INTERPRO_DOMAIN,DBNSFP_LRT_PRED,...,VEP_CADD_RAW,VEP_FEATURE,VEP_IMPACT,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_PROTEIN_POS,VEP_SIFT_SCORE,Variant Classification,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,


In [43]:
working_df = working_df.drop(columns='VEP_IMPACT')

##### Max MAF

In [44]:
working_df[['Max MAF (%)', 'Max MAF Source', 'OVERALL_MAX_MAF', 'OVERALL_MAX_MAF_SOURCE']]

,Max MAF (%),Max MAF Source,OVERALL_MAX_MAF,OVERALL_MAX_MAF_SOURCE
Genomic Description (GRCh37),,,,
20:21686358:T>C,0.000000,NaN,NaN,NaN
20:21686361:C>T,0.006660,gnomad_AF_nfe_pass,6.65956e-05,gnomad_AF_nfe_pass
20:21686379:G>A,0.010493,gnomad_AF_nfe_pass,0.000104932,gnomad_AF_nfe_pass
20:21686382:C>T,3.187450,gnomad_AF_afr_pass,0.0318745,gnomad_AF_afr_pass
20:21686390:G>T,0.000000,NaN,NaN,NaN
...,...,...,...,...
10:8115915:C>T,0.000000,NaN,[],[]
10:8115916:C>G,0.006484,gnomad_AF_nfe_pass,6.48424e-05,gnomad_AF_nfe_pass
10:8115916:C>T,0.012309,gnomad_AF_afr_pass,0.000123092,gnomad_AF_afr_pass


Everywhere that Max MAF Source does not match OVERALL_MAX_MAF_SOURCE also does not have any MAF data

Mismatches in Max MAF % are just rounding errors

In [45]:
temp_df = working_df.loc[working_df['Max MAF Source'] != working_df['OVERALL_MAX_MAF_SOURCE'], ['Max MAF (%)', 'Max MAF Source', 'OVERALL_MAX_MAF', 'OVERALL_MAX_MAF_SOURCE']]
temp_df.head()

,Max MAF (%),Max MAF Source,OVERALL_MAX_MAF,OVERALL_MAX_MAF_SOURCE
Genomic Description (GRCh37),,,,
20:21686358:T>C,0.0,NaN,NaN,NaN
20:21686390:G>T,0.0,NaN,NaN,NaN
20:21686398:G>T,0.0,NaN,NaN,NaN
20:21686443:C>A,0.0,NaN,NaN,NaN
20:21686443:C>G,0.0,NaN,NaN,NaN


In [46]:
temp_df = working_df.loc[working_df['Max MAF (%)'] != working_df['OVERALL_MAX_MAF'], ['Max MAF (%)', 'Max MAF Source', 'OVERALL_MAX_MAF', 'OVERALL_MAX_MAF_SOURCE']]
temp_df.head()

,Max MAF (%),Max MAF Source,OVERALL_MAX_MAF,OVERALL_MAX_MAF_SOURCE
Genomic Description (GRCh37),,,,
20:21686358:T>C,0.000000,NaN,NaN,NaN
20:21686361:C>T,0.006660,gnomad_AF_nfe_pass,6.65956e-05,gnomad_AF_nfe_pass
20:21686379:G>A,0.010493,gnomad_AF_nfe_pass,0.000104932,gnomad_AF_nfe_pass
20:21686382:C>T,3.187450,gnomad_AF_afr_pass,0.0318745,gnomad_AF_afr_pass
20:21686390:G>T,0.000000,NaN,NaN,NaN


In [47]:
working_df = working_df.drop(columns=['OVERALL_MAX_MAF', 'OVERALL_MAX_MAF_SOURCE'])

In [48]:
working_df.loc[working_df['Variant Classification'] != working_df['FINAL_PATHOGENICITY'], ['Variant Classification', 'FINAL_PATHOGENICITY']]

,Variant Classification,FINAL_PATHOGENICITY
Genomic Description (GRCh37),,


In [49]:
working_df = working_df.drop(columns='FINAL_PATHOGENICITY')

In [50]:
working_df.columns

Index(['CADD Phred', 'CLINVAR_CLNSIG', 'CLINVAR_PATHOGENIC',
       'CURATED_PATHOGENICITY', 'Confidence Score', 'Consequence',
       'DBNSFP_GERP_RS', 'DBNSFP_GERP_RS_PRED', 'DBNSFP_INTERPRO_DOMAIN',
       'DBNSFP_LRT_PRED', 'DBNSFP_LRT_SCORE', 'DBNSFP_MUTATIONTASTER_PRED',
       'DBNSFP_MUTATIONTASTER_SCORE', 'DBNSFP_PHYLOP30WAY_MAMMALIAN',
       'DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED', 'DBNSFP_POLYPHEN2_HDIV_PRED',
       'DBNSFP_POLYPHEN2_HDIV_SCORE', 'DBNSFP_SIFT_PRED', 'DBNSFP_SIFT_SCORE',
       'FINAL_PRED', 'GENE', 'Impact', 'Interpro Domain', 'Max MAF (%)',
       'Max MAF Source', 'NUM_PATH_PREDS', 'Normalized SA', 'Surface Area',
       'TOTAL_NUM_PREDS', 'TYPE', 'UNIPROT_DOMAIN', 'VEP_CADD_RAW',
       'VEP_FEATURE', 'VEP_POLYPHEN_PRED', 'VEP_POLYPHEN_SCORE',
       'VEP_PROTEIN_POS', 'VEP_SIFT_SCORE', 'Variant Classification', 'ddG',
       '|ddG|'],
      dtype='object')

### Filtering Instances

In [51]:
working_df = working_df[working_df.TYPE == 'SNV'].drop(columns='TYPE') # Drop multi-nucleotide variants (MNVS), remove TYPE column

In [52]:
working_df = working_df[working_df.Consequence == 'missense_variant'] # Drop missense variants with secondary effects, i.e., splite region variant

In [53]:
working_df = working_df[working_df['Max MAF (%)'] >= 1E-10] # Drop variants with MAF > 10^-10 => effectively 0

In [54]:
working_df.shape

(95354, 39)

In [55]:
working_df[working_df['Max MAF (%)'] != 0].shape

(95354, 39)

### Removing Homogenous Columns

In [56]:
print(working_df.Impact.value_counts())
working_df = working_df.drop(columns='Impact')

Impact
MODERATE    95354
Name: count, dtype: int64


In [57]:
print(working_df.Consequence.value_counts())
working_df = working_df.drop(columns=['Consequence'])

Consequence
missense_variant    95354
Name: count, dtype: int64


In [58]:
working_df.head()

,CADD Phred,CLINVAR_CLNSIG,CLINVAR_PATHOGENIC,CURATED_PATHOGENICITY,Confidence Score,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_INTERPRO_DOMAIN,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,...,UNIPROT_DOMAIN,VEP_CADD_RAW,VEP_FEATURE,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_PROTEIN_POS,VEP_SIFT_SCORE,Variant Classification,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686361:C>T,23.00,NaN,NaN,NaN,38.06,4.28,C,.&.,NaN,NaN,...,NaN,2.751772,NM_006192.5,P,0.917,4,0.00,Unknown significance,-0.0,0.0
20:21686379:G>A,22.90,NaN,NaN,NaN,37.23,2.32,C,.&.,NaN,NaN,...,NaN,2.662314,NM_006192.5,P,0.787,10,0.00,Unknown significance,-0.3,0.3
20:21686382:C>T,21.80,NaN,NaN,NaN,34.61,2.11,C,.&.,NaN,NaN,...,NaN,2.257539,NM_006192.5,P,0.804,11,0.01,Benign,-0.3,0.3
20:21686402:G>C,7.08,NaN,NaN,NaN,36.34,-0.29,N,.&.,NaN,NaN,...,NaN,0.289852,NM_006192.5,B,0.000,18,1.00,Likely benign,0.5,0.5
20:21686445:C>T,20.50,NaN,NaN,NaN,35.99,3.25,C,.&.,NaN,NaN,...,NaN,2.089529,NM_006192.5,P,0.947,32,0.00,Unknown significance,0.1,0.1


### Removing columns that are too varied

In [59]:
print(len(working_df.VEP_FEATURE.value_counts()))
working_df = working_df.drop(columns='VEP_FEATURE')

247


In [60]:
working_df = working_df.rename(columns={'Max MAF (%)': 'MAX_MAF', 'Max MAF Source': 'MAX_MAF_SOURCE'})

In [61]:
percentage_not_nan = working_df.notna().mean() * 100
percentage_not_nan

CADD Phred                           100.000000
CLINVAR_CLNSIG                        12.167292
CLINVAR_PATHOGENIC                    12.159951
CURATED_PATHOGENICITY                  0.328251
Confidence Score                     100.000000
DBNSFP_GERP_RS                        99.958051
DBNSFP_GERP_RS_PRED                   99.958051
DBNSFP_INTERPRO_DOMAIN                89.045032
DBNSFP_LRT_PRED                       87.532773
DBNSFP_LRT_SCORE                      87.532773
DBNSFP_MUTATIONTASTER_PRED            99.409569
DBNSFP_MUTATIONTASTER_SCORE           99.409569
DBNSFP_PHYLOP30WAY_MAMMALIAN          99.976928
DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED     99.976928
DBNSFP_POLYPHEN2_HDIV_PRED            99.419007
DBNSFP_POLYPHEN2_HDIV_SCORE           99.419007
DBNSFP_SIFT_PRED                      98.541225
DBNSFP_SIFT_SCORE                     98.541225
FINAL_PRED                            99.976928
GENE                                 100.000000
Interpro Domain                       40

In [62]:
working_df.loc[:, working_df.columns.str.contains('Domain') | working_df.columns.str.contains('DOMAIN')]

,DBNSFP_INTERPRO_DOMAIN,Interpro Domain,UNIPROT_DOMAIN
Genomic Description (GRCh37),,,
20:21686361:C>T,.&.,NaN,NaN
20:21686379:G>A,.&.,NaN,NaN
20:21686382:C>T,.&.,NaN,NaN
20:21686402:G>C,.&.,NaN,NaN
20:21686445:C>T,.&.,NaN,NaN
...,...,...,...
10:8115913:C>T,.&.&.,NaN,NaN
10:8115916:C>G,.&.&.,NaN,NaN
10:8115916:C>T,.&.&.,NaN,NaN


### Removing columns that are too sparse

['CLINVAR_CLNSIG', 'CLINVAR_PATHOGENIC', 'CURATED_PATHOGENICITY'] have values for fewer than a third of the variants

In [63]:
utils.difference(working_df.columns.tolist(), working_df.dropna(thresh=working_df.shape[0] / 3, axis=1).columns.tolist())

['CLINVAR_CLNSIG', 'CLINVAR_PATHOGENIC', 'CURATED_PATHOGENICITY']

In [64]:
working_df = working_df.dropna(thresh=working_df.shape[0] / 3, axis=1)

Renaming the columns that include spaces

In [65]:
working_df = working_df.rename(columns={'CADD Phred': 'CADD_PHRED', 'Confidence Score': 'CONFIDENCE_SCORE',
                           'Surface Area': 'SURFACE_AREA', 'Normalized SA': 'NORMALIZED_SURFACE_AREA',
                           'Interpro Domain': 'INTERPRO_DOMAIN', 'Variant Classification': 'VARIANT_CLASSIFICATION'})

I'm going to ignore unraveling the complicated prediction columns for now

In [66]:
working_df

,CADD_PHRED,CONFIDENCE_SCORE,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_INTERPRO_DOMAIN,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_PHYLOP30WAY_MAMMALIAN,...,TOTAL_NUM_PREDS,UNIPROT_DOMAIN,VEP_CADD_RAW,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_PROTEIN_POS,VEP_SIFT_SCORE,VARIANT_CLASSIFICATION,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686361:C>T,23.00,38.06,4.28,C,.&.,NaN,NaN,N,0.758794,0.118,...,5.0,NaN,2.751772,P,0.917,4,0.00,Unknown significance,-0.0,0.0
20:21686379:G>A,22.90,37.23,2.32,C,.&.,NaN,NaN,N,0.999272,1.147,...,5.0,NaN,2.662314,P,0.787,10,0.00,Unknown significance,-0.3,0.3
20:21686382:C>T,21.80,34.61,2.11,C,.&.,NaN,NaN,N,0.998176,0.106,...,5.0,NaN,2.257539,P,0.804,11,0.01,Benign,-0.3,0.3
20:21686402:G>C,7.08,36.34,-0.29,N,.&.,NaN,NaN,N,1,0.111,...,5.0,NaN,0.289852,B,0.000,18,1.00,Likely benign,0.5,0.5
20:21686445:C>T,20.50,35.99,3.25,C,.&.,NaN,NaN,N,0.995526,0.937,...,5.0,NaN,2.089529,P,0.947,32,0.00,Unknown significance,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115913:C>T,28.40,51.75,5.59,C,.&.&.,D,0.000000,D&D,1&1,1.026,...,6.0,NaN,4.074794,D,1.000,421,0.01,Likely pathogenic,-0.1,0.1
10:8115916:C>G,27.50,47.29,5.59,C,.&.&.,U,0.000643,D&D,1&1,1.026,...,5.0,NaN,3.979789,D,1.000,422,0.00,Unknown significance,-0.3,0.3
10:8115916:C>T,29.20,47.29,5.59,C,.&.&.,U,0.000643,D&D,1&1,1.026,...,5.0,NaN,4.163516,D,1.000,422,0.00,Unknown significance,-0.2,0.2


In [67]:
working_df.columns

Index(['CADD_PHRED', 'CONFIDENCE_SCORE', 'DBNSFP_GERP_RS',
       'DBNSFP_GERP_RS_PRED', 'DBNSFP_INTERPRO_DOMAIN', 'DBNSFP_LRT_PRED',
       'DBNSFP_LRT_SCORE', 'DBNSFP_MUTATIONTASTER_PRED',
       'DBNSFP_MUTATIONTASTER_SCORE', 'DBNSFP_PHYLOP30WAY_MAMMALIAN',
       'DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED', 'DBNSFP_POLYPHEN2_HDIV_PRED',
       'DBNSFP_POLYPHEN2_HDIV_SCORE', 'DBNSFP_SIFT_PRED', 'DBNSFP_SIFT_SCORE',
       'FINAL_PRED', 'GENE', 'INTERPRO_DOMAIN', 'MAX_MAF', 'MAX_MAF_SOURCE',
       'NUM_PATH_PREDS', 'NORMALIZED_SURFACE_AREA', 'SURFACE_AREA',
       'TOTAL_NUM_PREDS', 'UNIPROT_DOMAIN', 'VEP_CADD_RAW',
       'VEP_POLYPHEN_PRED', 'VEP_POLYPHEN_SCORE', 'VEP_PROTEIN_POS',
       'VEP_SIFT_SCORE', 'VARIANT_CLASSIFICATION', 'ddG', '|ddG|'],
      dtype='object')

In [68]:
(working_df['VARIANT_CLASSIFICATION'] == working_df['FINAL_PRED']).value_counts()

True     88539
False     6815
Name: count, dtype: int64

In [69]:
working_df = working_df.drop(columns='VEP_PROTEIN_POS')

# Preproccesing

## Encoding Columns

In [70]:
df = working_df.copy()
df.head()

,CADD_PHRED,CONFIDENCE_SCORE,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_INTERPRO_DOMAIN,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_PHYLOP30WAY_MAMMALIAN,...,SURFACE_AREA,TOTAL_NUM_PREDS,UNIPROT_DOMAIN,VEP_CADD_RAW,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_SIFT_SCORE,VARIANT_CLASSIFICATION,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686361:C>T,23.00,38.06,4.28,C,.&.,NaN,NaN,N,0.758794,0.118,...,139.065932,5.0,NaN,2.751772,P,0.917,0.00,Unknown significance,-0.0,0.0
20:21686379:G>A,22.90,37.23,2.32,C,.&.,NaN,NaN,N,0.999272,1.147,...,180.703642,5.0,NaN,2.662314,P,0.787,0.00,Unknown significance,-0.3,0.3
20:21686382:C>T,21.80,34.61,2.11,C,.&.,NaN,NaN,N,0.998176,0.106,...,76.425233,5.0,NaN,2.257539,P,0.804,0.01,Benign,-0.3,0.3
20:21686402:G>C,7.08,36.34,-0.29,N,.&.,NaN,NaN,N,1,0.111,...,68.153896,5.0,NaN,0.289852,B,0.000,1.00,Likely benign,0.5,0.5
20:21686445:C>T,20.50,35.99,3.25,C,.&.,NaN,NaN,N,0.995526,0.937,...,72.826957,5.0,NaN,2.089529,P,0.947,0.00,Unknown significance,0.1,0.1


In [71]:
df.shape

(95354, 32)

In [72]:
df2 = df.copy()

In [73]:
[(col, 'predictions') if 'PRED' in col else (col, 'score') if 'SCORE' in col else (col, 'other') for col in df.columns]

[('CADD_PHRED', 'other'),
 ('CONFIDENCE_SCORE', 'score'),
 ('DBNSFP_GERP_RS', 'other'),
 ('DBNSFP_GERP_RS_PRED', 'predictions'),
 ('DBNSFP_INTERPRO_DOMAIN', 'other'),
 ('DBNSFP_LRT_PRED', 'predictions'),
 ('DBNSFP_LRT_SCORE', 'score'),
 ('DBNSFP_MUTATIONTASTER_PRED', 'predictions'),
 ('DBNSFP_MUTATIONTASTER_SCORE', 'score'),
 ('DBNSFP_PHYLOP30WAY_MAMMALIAN', 'other'),
 ('DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED', 'predictions'),
 ('DBNSFP_POLYPHEN2_HDIV_PRED', 'predictions'),
 ('DBNSFP_POLYPHEN2_HDIV_SCORE', 'score'),
 ('DBNSFP_SIFT_PRED', 'predictions'),
 ('DBNSFP_SIFT_SCORE', 'score'),
 ('FINAL_PRED', 'predictions'),
 ('GENE', 'other'),
 ('INTERPRO_DOMAIN', 'other'),
 ('MAX_MAF', 'other'),
 ('MAX_MAF_SOURCE', 'other'),
 ('NUM_PATH_PREDS', 'predictions'),
 ('NORMALIZED_SURFACE_AREA', 'other'),
 ('SURFACE_AREA', 'other'),
 ('TOTAL_NUM_PREDS', 'predictions'),
 ('UNIPROT_DOMAIN', 'other'),
 ('VEP_CADD_RAW', 'other'),
 ('VEP_POLYPHEN_PRED', 'predictions'),
 ('VEP_POLYPHEN_SCORE', 'score'),
 ('VE

In [74]:
cadd = ['CADD_PHRED', 'VEP_CADD_RAW']
classification = ['VARIANT_CLASSIFICATION', 'FINAL_PRED', 'NUM_PATH_PREDS', 'TOTAL_NUM_PREDS']
gene = ['GENE']
maf = ['MAX_MAF', 'MAX_MAF_SOURCE']
domain = ['DBNSFP_INTERPRO_DOMAIN', 'UNIPROT_DOMAIN', 'INTERPRO_DOMAIN']
score = ['DBNSFP_GERP_RS', 'DBNSFP_LRT_SCORE', 'DBNSFP_MUTATIONTASTER_SCORE', 'DBNSFP_POLYPHEN2_HDIV_SCORE', 'DBNSFP_SIFT_SCORE', 'DBNSFP_PHYLOP30WAY_MAMMALIAN', 'VEP_POLYPHEN_SCORE', 'VEP_SIFT_SCORE']
predictions = ['DBNSFP_SIFT_PRED', 'DBNSFP_GERP_RS_PRED', 'DBNSFP_LRT_PRED', 'DBNSFP_MUTATIONTASTER_PRED', 'DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED', 'DBNSFP_POLYPHEN2_HDIV_PRED', 'VEP_POLYPHEN_PRED']
protein_structure = ['ddG', '|ddG|', 'CONFIDENCE_SCORE', 'SURFACE_AREA', 'NORMALIZED_SURFACE_AREA']

In [75]:
df2 = df.copy()
df2.columns = pd.MultiIndex.from_tuples([(col, 'predictions') if 'PRED' in col else (col, 'score') if 'SCORE' in col else (col, 'other') for col in df.columns])
df2

,CADD_PHRED,CONFIDENCE_SCORE,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_INTERPRO_DOMAIN,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_PHYLOP30WAY_MAMMALIAN,...,SURFACE_AREA,TOTAL_NUM_PREDS,UNIPROT_DOMAIN,VEP_CADD_RAW,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_SIFT_SCORE,VARIANT_CLASSIFICATION,ddG,|ddG|
,other,score,other,predictions,other,predictions,score,predictions,score,other,...,other,predictions,other,other,predictions,score,score,other,other,other
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686361:C>T,23.00,38.06,4.28,C,.&.,NaN,NaN,N,0.758794,0.118,...,139.065932,5.0,NaN,2.751772,P,0.917,0.00,Unknown significance,-0.0,0.0
20:21686379:G>A,22.90,37.23,2.32,C,.&.,NaN,NaN,N,0.999272,1.147,...,180.703642,5.0,NaN,2.662314,P,0.787,0.00,Unknown significance,-0.3,0.3
20:21686382:C>T,21.80,34.61,2.11,C,.&.,NaN,NaN,N,0.998176,0.106,...,76.425233,5.0,NaN,2.257539,P,0.804,0.01,Benign,-0.3,0.3
20:21686402:G>C,7.08,36.34,-0.29,N,.&.,NaN,NaN,N,1,0.111,...,68.153896,5.0,NaN,0.289852,B,0.000,1.00,Likely benign,0.5,0.5
20:21686445:C>T,20.50,35.99,3.25,C,.&.,NaN,NaN,N,0.995526,0.937,...,72.826957,5.0,NaN,2.089529,P,0.947,0.00,Unknown significance,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115913:C>T,28.40,51.75,5.59,C,.&.&.,D,0.000000,D&D,1&1,1.026,...,107.686086,6.0,NaN,4.074794,D,1.000,0.01,Likely pathogenic,-0.1,0.1
10:8115916:C>G,27.50,47.29,5.59,C,.&.&.,U,0.000643,D&D,1&1,1.026,...,136.946574,5.0,NaN,3.979789,D,1.000,0.00,Unknown significance,-0.3,0.3


In [76]:
other_columns = df2.xs('other', axis=1, level=1)
print(other_columns)


                              CADD_PHRED  DBNSFP_GERP_RS  \
Genomic Description (GRCh37)                               
20:21686361:C>T                    23.00            4.28   
20:21686379:G>A                    22.90            2.32   
20:21686382:C>T                    21.80            2.11   
20:21686402:G>C                     7.08           -0.29   
20:21686445:C>T                    20.50            3.25   
...                                  ...             ...   
10:8115913:C>T                     28.40            5.59   
10:8115916:C>G                     27.50            5.59   
10:8115916:C>T                     29.20            5.59   
10:8115927:C>T                     14.78            3.68   
10:8115945:G>A                     27.60            5.59   

                             DBNSFP_INTERPRO_DOMAIN  \
Genomic Description (GRCh37)                          
20:21686361:C>T                                 .&.   
20:21686379:G>A                                 .&.   

In [77]:
df_copy = df.copy()

In [78]:
df = df_copy.copy()

In [79]:
df = df.drop(columns=['DBNSFP_INTERPRO_DOMAIN', 'INTERPRO_DOMAIN', 'UNIPROT_DOMAIN'])
df

,CADD_PHRED,CONFIDENCE_SCORE,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_PHYLOP30WAY_MAMMALIAN,DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED,...,NORMALIZED_SURFACE_AREA,SURFACE_AREA,TOTAL_NUM_PREDS,VEP_CADD_RAW,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_SIFT_SCORE,VARIANT_CLASSIFICATION,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686361:C>T,23.00,38.06,4.28,C,NaN,NaN,N,0.758794,0.118,N,...,0.883870,139.065932,5.0,2.751772,P,0.917,0.00,Unknown significance,-0.0,0.0
20:21686379:G>A,22.90,37.23,2.32,C,NaN,NaN,N,0.999272,1.147,C,...,0.670700,180.703642,5.0,2.662314,P,0.787,0.00,Unknown significance,-0.3,0.3
20:21686382:C>T,21.80,34.61,2.11,C,NaN,NaN,N,0.998176,0.106,N,...,0.601022,76.425233,5.0,2.257539,P,0.804,0.01,Benign,-0.3,0.3
20:21686402:G>C,7.08,36.34,-0.29,N,NaN,NaN,N,1,0.111,N,...,0.714805,68.153896,5.0,0.289852,B,0.000,1.00,Likely benign,0.5,0.5
20:21686445:C>T,20.50,35.99,3.25,C,NaN,NaN,N,0.995526,0.937,N,...,0.572725,72.826957,5.0,2.089529,P,0.947,0.00,Unknown significance,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115913:C>T,28.40,51.75,5.59,C,D,0.000000,D&D,1&1,1.026,C,...,0.684427,107.686086,6.0,4.074794,D,1.000,0.01,Likely pathogenic,-0.1,0.1
10:8115916:C>G,27.50,47.29,5.59,C,U,0.000643,D&D,1&1,1.026,C,...,0.870607,136.946574,5.0,3.979789,D,1.000,0.00,Unknown significance,-0.3,0.3
10:8115916:C>T,29.20,47.29,5.59,C,U,0.000643,D&D,1&1,1.026,C,...,0.870607,136.946574,5.0,4.163516,D,1.000,0.00,Unknown significance,-0.2,0.2


HERE

In [80]:
df['PREDICTION_RATIO'] = df['NUM_PATH_PREDS'] / df['TOTAL_NUM_PREDS']
df['INTERNAL_STATE'] = 'VUS'
df.loc[df['TOTAL_NUM_PREDS'] >= 5, 'INTERNAL_STATE'] = 'D'
df.loc[(df['TOTAL_NUM_PREDS'] >= 5) & (df['PREDICTION_RATIO'] < 0.6), 'INTERNAL_STATE'] = 'B'
df

,CADD_PHRED,CONFIDENCE_SCORE,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_PHYLOP30WAY_MAMMALIAN,DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED,...,TOTAL_NUM_PREDS,VEP_CADD_RAW,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_SIFT_SCORE,VARIANT_CLASSIFICATION,ddG,|ddG|,PREDICTION_RATIO,INTERNAL_STATE
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686361:C>T,23.00,38.06,4.28,C,NaN,NaN,N,0.758794,0.118,N,...,5.0,2.751772,P,0.917,0.00,Unknown significance,-0.0,0.0,0.600000,D
20:21686379:G>A,22.90,37.23,2.32,C,NaN,NaN,N,0.999272,1.147,C,...,5.0,2.662314,P,0.787,0.00,Unknown significance,-0.3,0.3,0.800000,D
20:21686382:C>T,21.80,34.61,2.11,C,NaN,NaN,N,0.998176,0.106,N,...,5.0,2.257539,P,0.804,0.01,Benign,-0.3,0.3,0.600000,D
20:21686402:G>C,7.08,36.34,-0.29,N,NaN,NaN,N,1,0.111,N,...,5.0,0.289852,B,0.000,1.00,Likely benign,0.5,0.5,0.000000,B
20:21686445:C>T,20.50,35.99,3.25,C,NaN,NaN,N,0.995526,0.937,N,...,5.0,2.089529,P,0.947,0.00,Unknown significance,0.1,0.1,0.600000,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115913:C>T,28.40,51.75,5.59,C,D,0.000000,D&D,1&1,1.026,C,...,6.0,4.074794,D,1.000,0.01,Likely pathogenic,-0.1,0.1,1.000000,D
10:8115916:C>G,27.50,47.29,5.59,C,U,0.000643,D&D,1&1,1.026,C,...,5.0,3.979789,D,1.000,0.00,Unknown significance,-0.3,0.3,1.000000,D
10:8115916:C>T,29.20,47.29,5.59,C,U,0.000643,D&D,1&1,1.026,C,...,5.0,4.163516,D,1.000,0.00,Unknown significance,-0.2,0.2,1.000000,D


In [89]:
df.loc[:, ['VARIANT_CLASSIFICATION', 'FINAL_PRED']] = df[['VARIANT_CLASSIFICATION', 'FINAL_PRED']].replace({'Unknown significance': 'VUS', 'Likely benign': 'LB', 'Likely pathogenic': 'LB', 'Benign': 'B', 'Pathogenic': 'P'})

In [90]:
final_classifications_df = df[['VARIANT_CLASSIFICATION', 'FINAL_PRED', 'NUM_PATH_PREDS', 'TOTAL_NUM_PREDS', 'PREDICTION_RATIO', 'INTERNAL_STATE']]
final_classifications_df

,VARIANT_CLASSIFICATION,FINAL_PRED,NUM_PATH_PREDS,TOTAL_NUM_PREDS,PREDICTION_RATIO,INTERNAL_STATE
Genomic Description (GRCh37),,,,,,
20:21686361:C>T,VUS,VUS,3.0,5.0,0.600000,D
20:21686379:G>A,VUS,VUS,4.0,5.0,0.800000,D
20:21686382:C>T,B,VUS,3.0,5.0,0.600000,D
20:21686402:G>C,LB,LB,0.0,5.0,0.000000,B
20:21686445:C>T,VUS,VUS,3.0,5.0,0.600000,D
...,...,...,...,...,...,...
10:8115913:C>T,LB,VUS,6.0,6.0,1.000000,D
10:8115916:C>G,VUS,VUS,5.0,5.0,1.000000,D
10:8115916:C>T,VUS,VUS,5.0,5.0,1.000000,D


Only 3419 missense variats from the ddG data that are labeled either pathogenic or benign

In [92]:
definitive_classification_df = final_classifications_df[final_classifications_df['VARIANT_CLASSIFICATION'].isin(['P', 'B'])]
print(definitive_classification_df.shape[0]/final_classifications_df.shape[0])
definitive_classification_df

0.035855863414224885


,VARIANT_CLASSIFICATION,FINAL_PRED,NUM_PATH_PREDS,TOTAL_NUM_PREDS,PREDICTION_RATIO,INTERNAL_STATE
Genomic Description (GRCh37),,,,,,
20:21686382:C>T,B,VUS,3.0,5.0,0.600000,D
20:21686448:T>C,B,VUS,4.0,5.0,0.800000,D
20:21686456:C>G,B,LB,1.0,5.0,0.200000,B
20:21687286:G>T,P,VUS,6.0,6.0,1.000000,D
20:21689216:G>A,P,LB,2.0,6.0,0.333333,B
...,...,...,...,...,...,...
10:73574874:G>A,P,VUS,6.0,6.0,1.000000,D
10:8100506:C>G,B,LB,2.0,6.0,0.333333,B
10:8100763:G>C,B,VUS,4.0,6.0,0.666667,D


In [95]:
definitive_classification_df[['VARIANT_CLASSIFICATION', 'INTERNAL_STATE']].value_counts()

VARIANT_CLASSIFICATION  INTERNAL_STATE
P                       D                 1584
B                       B                  937
                        D                  618
P                       B                  226
B                       VUS                 52
P                       VUS                  2
Name: count, dtype: int64

In [93]:
from sklearn.metrics import accuracy_score

accuracy_score(definitive_classification_df.VARIANT_CLASSIFICATION, definitive_classification_df.INTERNAL_STATE.replace('D', 'P'))

0.7373501023691138

In [82]:
df = pd.concat(dfs).map(utils.safe_decode)
df

/var/folders/3f/85zthwnn17dfrdm_swynn0qr0000gn/T/ipykernel_68115/2360911525.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs).map(utils.safe_decode)


,TYPE,GENE,gnomad_MAX_MAF,gnomad_MAX_MAF_SOURCE,gnomad_AC_afr_pass,gnomad_AC_amr_pass,gnomad_AC_eas_pass,gnomad_AC_nfe_pass,gnomad_AN_afr_pass,gnomad_AN_amr_pass,...,Max MAF Source,Surface Area,Normalized SA,Confidence Score,ddG,|ddG|,ca...,in...,calc...,second part
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686358:T>C,SNV,PAX1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,172.951510,0.752780,32.73,-1.4,1.4,NaN,NaN,NaN,NaN
20:21686361:C>T,SNV,PAX1,0.000067,gnomad_AF_nfe_pass,0.0,0.0,0.0,1.0,8602.0,810.0,...,gnomad_AF_nfe_pass,139.065932,0.883870,38.06,-0.0,0.0,NaN,NaN,NaN,NaN
20:21686379:G>A,SNV,PAX1,0.000105,gnomad_AF_nfe_pass,0.0,0.0,0.0,1.0,1492.0,274.0,...,gnomad_AF_nfe_pass,180.703642,0.670700,37.23,-0.3,0.3,NaN,NaN,NaN,NaN
20:21686382:C>T,SNV,PAX1,0.031875,gnomad_AF_afr_pass,319.0,1.0,0.0,4.0,10008.0,1046.0,...,gnomad_AF_afr_pass,76.425233,0.601022,34.61,-0.3,0.3,NaN,NaN,NaN,NaN
20:21686390:G>T,SNV,PAX1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,99.401913,0.578281,31.28,-0.6,0.6,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8115915:C>T,SNV,GATA3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,136.946574,0.870607,47.29,-0.3,0.3,NaN,NaN,NaN,NaN
10:8115916:C>G,SNV,GATA3,0.000065,gnomad_AF_nfe_pass,0.0,0.0,0.0,1.0,8702.0,848.0,...,gnomad_AF_nfe_pass,136.946574,0.870607,47.29,-0.3,0.3,NaN,NaN,NaN,NaN
10:8115916:C>T,SNV,GATA3,0.000123,gnomad_AF_afr_pass,2.0,1.0,0.0,2.0,16248.0,34588.0,...,gnomad_AF_afr_pass,136.946574,0.870607,47.29,-0.2,0.2,NaN,NaN,NaN,NaN


In [83]:
definitively_benign_or_pathogenic = df.loc[df.FINAL_PATHOGENICITY.isin(['Benign', 'Pathogenic']), :]
print(definitively_benign_or_pathogenic.shape[0])

8022


In [84]:
definitively_benign_or_pathogenic_missense = definitively_benign_or_pathogenic[definitively_benign_or_pathogenic.VEP_CONSEQUENCE == 'missense_variant']
print(definitively_benign_or_pathogenic_missense.shape[0])

7551


In [85]:
definitively_benign_or_pathogenic_missense = definitively_benign_or_pathogenic_missense.loc[:, ['FINAL_PATHOGENICITY', 'FINAL_PRED', 'NUM_PATH_PREDS', 'TOTAL_NUM_PREDS']]

In [86]:
definitively_benign_or_pathogenic_missense.loc[:, ['FINAL_PATHOGENICITY', 'FINAL_PRED']].value_counts()

FINAL_PATHOGENICITY  FINAL_PRED          
Pathogenic           Unknown significance    5665
Benign               Unknown significance     851
                     Likely benign            772
Pathogenic           Likely benign            249
Name: count, dtype: int64

In [87]:
definitive_classification_df.loc[:, ['VARIANT_CLASSIFICATION', 'FINAL_PRED']].value_counts()

VARIANT_CLASSIFICATION  FINAL_PRED          
Pathogenic              Unknown significance    1673
Benign                  Unknown significance     839
                        Likely benign            768
Pathogenic              Likely benign            139
Name: count, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder().fit(variants_w_label_df['VARIANT_CLASSIFICATION'])

variants_w_label_df.loc[:, 'VARIANT_CLASSIFICATION'] = le.transform(variants_w_label_df.loc[:, 'VARIANT_CLASSIFICATION'])
variants_w_label_df

,CADD_PHRED,CONFIDENCE_SCORE,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_PHYLOP30WAY_MAMMALIAN,DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED,...,NORMALIZED_SURFACE_AREA,SURFACE_AREA,TOTAL_NUM_PREDS,VEP_CADD_RAW,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_SIFT_SCORE,VARIANT_CLASSIFICATION,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686382:C>T,21.800,34.61,2.110,C,NaN,NaN,N,0.998176,0.106,N,...,0.601022,76.425233,5.0,2.257539,P,0.804,0.01,0,-0.3,0.3
20:21686402:G>C,7.080,36.34,-0.290,N,NaN,NaN,N,1,0.111,N,...,0.714805,68.153896,5.0,0.289852,B,0.000,1.00,2,0.5,0.5
20:21686448:T>C,16.360,32.99,0.193,C,NaN,NaN,D,0.994473,1.037,C,...,0.918470,177.128056,5.0,1.515567,B,0.001,0.00,0,-0.3,0.3
20:21686456:C>G,6.810,33.33,0.668,C,NaN,NaN,N,1,-0.295,N,...,0.821205,221.253638,5.0,0.269085,B,0.000,0.57,0,-0.1,0.1
20:21686459:G>A,0.005,33.78,-8.940,N,NaN,NaN,N,1,-3.370,N,...,0.455428,57.911682,5.0,-0.885486,B,0.052,0.28,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8111473:G>C,28.400,96.95,5.210,C,D,0.000000,D&D,1&1,1.176,C,...,0.023200,3.585018,6.0,4.083778,D,1.000,0.01,4,-4.5,4.5
10:8115817:A>C,19.950,36.58,3.240,C,N,0.000861,D&D,0.988413&0.988413,-0.339,N,...,1.046368,167.237965,6.0,2.027122,B,0.000,0.15,2,0.0,0.0
10:8115817:A>G,15.350,36.58,3.240,C,N,0.000861,D&D,0.961623&0.961623,-0.339,N,...,1.046368,167.237965,6.0,1.330430,B,0.000,1.00,2,0.0,0.0


In [ ]:
import numpy as np

variants_w_label_df.loc[:, 'VARIANT_CLASSIFICATION'] = np.where(variants_w_label_df['VARIANT_CLASSIFICATION'].str.contains('benign', case=False), 'benign', variants_w_label_df['VARIANT_CLASSIFICATION'])
variants_w_label_df.loc[:, 'VARIANT_CLASSIFICATION'] = np.where(variants_w_label_df['VARIANT_CLASSIFICATION'].str.contains('pathogenic', case=False), 'pathogenic', variants_w_label_df['VARIANT_CLASSIFICATION'])
variants_w_label_df.VARIANT_CLASSIFICATION.value_counts()

AttributeError: Can only use .str accessor with string values!

Remove variants where ddG data is unavailable

In [90]:
variants_w_label_df = variants_w_label_df[~variants_w_label_df.ddG.isna()]

NameError: name 'variants_w_label_df' is not defined

In [91]:
variants_w_label_df.VARIANT_CLASSIFICATION.value_counts()

NameError: name 'variants_w_label_df' is not defined

In [466]:
variants_w_label_df[classification + predictions]

,VARIANT_CLASSIFICATION,FINAL_PRED,NUM_PATH_PREDS,TOTAL_NUM_PREDS,DBNSFP_SIFT_PRED,DBNSFP_GERP_RS_PRED,DBNSFP_LRT_PRED,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED,DBNSFP_POLYPHEN2_HDIV_PRED,VEP_POLYPHEN_PRED
Genomic Description (GRCh37),,,,,,,,,,,
20:21686382:C>T,Benign,Unknown significance,3.0,5.0,D&.,C,NaN,N,N,P&.,P
20:21686402:G>C,Likely benign,Likely benign,0.0,5.0,T&.,N,NaN,N,N,B&.,B
20:21686448:T>C,Benign,Unknown significance,4.0,5.0,T&.,C,NaN,D,C,B&.,B
20:21686456:C>G,Benign,Likely benign,1.0,5.0,T&.,C,NaN,N,N,B&.,B
20:21686459:G>A,Likely benign,Likely benign,0.0,5.0,T&.,N,NaN,N,N,B&.,B
...,...,...,...,...,...,...,...,...,...,...,...
10:8111473:G>C,Pathogenic,Unknown significance,6.0,6.0,D&.&D,C,D,D&D,C,D&D&D,D
10:8115817:A>C,Likely benign,Likely benign,2.0,6.0,T&.&T,C,N,D&D,N,B&B&B,B
10:8115817:A>G,Likely benign,Likely benign,2.0,6.0,T&.&T,C,N,D&D,N,B&B&B,B


Split the columns that have list-like values into lists

In [467]:
ampersand_counts = variants_w_label_df.map(lambda x: '&' in str(x)).sum()
print(ampersand_counts)

CADD_PHRED                               0
CONFIDENCE_SCORE                         0
DBNSFP_GERP_RS                           0
DBNSFP_GERP_RS_PRED                      0
DBNSFP_LRT_PRED                          0
DBNSFP_LRT_SCORE                         0
DBNSFP_MUTATIONTASTER_PRED           20359
DBNSFP_MUTATIONTASTER_SCORE          20359
DBNSFP_PHYLOP30WAY_MAMMALIAN             0
DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED        0
DBNSFP_POLYPHEN2_HDIV_PRED           21569
DBNSFP_POLYPHEN2_HDIV_SCORE          21569
DBNSFP_SIFT_PRED                     21569
DBNSFP_SIFT_SCORE                    21569
FINAL_PRED                               0
GENE                                     0
MAX_MAF                                  0
MAX_MAF_SOURCE                           0
NUM_PATH_PREDS                           0
NORMALIZED_SURFACE_AREA                  0
SURFACE_AREA                             0
TOTAL_NUM_PREDS                          0
VEP_CADD_RAW                             0
VEP_POLYPHE

In [383]:
cols_w_ampersands = ampersand_counts[ampersand_counts > 0].index.tolist()
for col in cols_w_ampersands:
    variants_w_label_df.loc[:, col] = variants_w_label_df[col].astype('string').str.split('&')
variants_w_label_df[cols_w_ampersands]

,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_POLYPHEN2_HDIV_PRED,DBNSFP_POLYPHEN2_HDIV_SCORE,DBNSFP_SIFT_PRED,DBNSFP_SIFT_SCORE
Genomic Description (GRCh37),,,,,,
20:21686382:C>T,[N],[0.998176],"[P, .]","[0.804, .]","[D, .]","[0.002, .]"
20:21686402:G>C,[N],[1],"[B, .]","[0.0, .]","[T, .]","[0.436, .]"
20:21686448:T>C,[D],[0.994473],"[B, .]","[0.002, .]","[T, .]","[0.065, .]"
20:21686456:C>G,[N],[1],"[B, .]","[0.0, .]","[T, .]","[0.884, .]"
20:21686459:G>A,[N],[1],"[B, .]","[0.052, .]","[T, .]","[0.063, .]"
...,...,...,...,...,...,...
10:8111473:G>C,"[D, D]","[1, 1]","[D, D, D]","[1.0, 1.0, 1.0]","[D, ., D]","[0.0, ., 0.0]"
10:8115817:A>C,"[D, D]","[0.988413, 0.988413]","[B, B, B]","[0.0, 0.0, 0.0]","[T, ., T]","[0.354, ., 0.352]"
10:8115817:A>G,"[D, D]","[0.961623, 0.961623]","[B, B, B]","[0.0, 0.0, 0.0]","[T, ., T]","[1.0, ., 1.0]"


In [384]:
variants_w_label_df

,CADD_PHRED,CONFIDENCE_SCORE,DBNSFP_GERP_RS,DBNSFP_GERP_RS_PRED,DBNSFP_LRT_PRED,DBNSFP_LRT_SCORE,DBNSFP_MUTATIONTASTER_PRED,DBNSFP_MUTATIONTASTER_SCORE,DBNSFP_PHYLOP30WAY_MAMMALIAN,DBNSFP_PHYLOP30WAY_MAMMALIAN_PRED,...,NORMALIZED_SURFACE_AREA,SURFACE_AREA,TOTAL_NUM_PREDS,VEP_CADD_RAW,VEP_POLYPHEN_PRED,VEP_POLYPHEN_SCORE,VEP_SIFT_SCORE,VARIANT_CLASSIFICATION,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,,,,,,,,,,,,,,
20:21686382:C>T,21.800,34.61,2.110,C,NaN,NaN,[N],[0.998176],0.106,N,...,0.601022,76.425233,5.0,2.257539,P,0.804,0.01,0,-0.3,0.3
20:21686402:G>C,7.080,36.34,-0.290,N,NaN,NaN,[N],[1],0.111,N,...,0.714805,68.153896,5.0,0.289852,B,0.000,1.00,0,0.5,0.5
20:21686448:T>C,16.360,32.99,0.193,C,NaN,NaN,[D],[0.994473],1.037,C,...,0.918470,177.128056,5.0,1.515567,B,0.001,0.00,0,-0.3,0.3
20:21686456:C>G,6.810,33.33,0.668,C,NaN,NaN,[N],[1],-0.295,N,...,0.821205,221.253638,5.0,0.269085,B,0.000,0.57,0,-0.1,0.1
20:21686459:G>A,0.005,33.78,-8.940,N,NaN,NaN,[N],[1],-3.370,N,...,0.455428,57.911682,5.0,-0.885486,B,0.052,0.28,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10:8111473:G>C,28.400,96.95,5.210,C,D,0.000000,"[D, D]","[1, 1]",1.176,C,...,0.023200,3.585018,6.0,4.083778,D,1.000,0.01,1,-4.5,4.5
10:8115817:A>C,19.950,36.58,3.240,C,N,0.000861,"[D, D]","[0.988413, 0.988413]",-0.339,N,...,1.046368,167.237965,6.0,2.027122,B,0.000,0.15,0,0.0,0.0
10:8115817:A>G,15.350,36.58,3.240,C,N,0.000861,"[D, D]","[0.961623, 0.961623]",-0.339,N,...,1.046368,167.237965,6.0,1.330430,B,0.000,1.00,0,0.0,0.0


In [386]:
final_classifications_df = variants_w_label_df[['VARIANT_CLASSIFICATION', 'FINAL_PRED', 'NUM_PATH_PREDS', 'TOTAL_NUM_PREDS']]
final_classifications_df

,VARIANT_CLASSIFICATION,FINAL_PRED,NUM_PATH_PREDS,TOTAL_NUM_PREDS
Genomic Description (GRCh37),,,,
20:21686382:C>T,0,Unknown significance,3.0,5.0
20:21686402:G>C,0,Likely benign,0.0,5.0
20:21686448:T>C,0,Unknown significance,4.0,5.0
20:21686456:C>G,0,Likely benign,1.0,5.0
20:21686459:G>A,0,Likely benign,0.0,5.0
...,...,...,...,...
10:8111473:G>C,1,Unknown significance,6.0,6.0
10:8115817:A>C,0,Likely benign,2.0,6.0
10:8115817:A>G,0,Likely benign,2.0,6.0


In [385]:
from sklearn.metrics import accuracy_score

curated_classification = variants_w_label_df.VARIANT_CLASSIFICATION
azaiez_classification = variants_w_label_df.FINAL_PRED
majority_vote_classification = variants_w_label_df.NUM_PATH_PREDS / variants_w_label_df.TOTAL_NUM_PREDS

accuracy_score(curated_classification, azaiez_classification)

TypeError: '<' not supported between instances of 'float' and 'str'

# Model Building

In [121]:
from sklearn import feature_selection
from sklearn import feature_extraction
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [56]:
y = raw_data['Class']
X = raw_data.drop('Class',axis=1)

imputer = SimpleImputer(strategy='constant',fill_value=0, missing_values=np.nan)
imputer = imputer.fit(X[['CADD Phred','MAF','Surface Area','Normalized SA','Confidence Score','ddG']])
X[['CADD Phred','MAF','Surface Area','Normalized SA','Confidence Score','ddG']]=imputer.transform(X[['CADD Phred','MAF','Surface Area','Normalized SA','Confidence Score','ddG']])

NameError: name 'raw_data' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear',probability=True)
svclassifier.fit(X_train, y_train)

y_pred_prob = svclassifier.predict_proba(X_test)
y_pred_prob
y_pred=svclassifier.predict(X_test)
display(pd.DataFrame({'svclassifier.predict': y_pred,
                     'svclassifier.predict_prob_a[:,0] (Pathogenic)': y_pred_prob[:,0],
                     'svclassifier.predict_prob_a[:,1] (Non-Pathogenic)': y_pred_prob[:,1]}).head(100))
y_predictions_prob_a = y_pred_prob[:,0]

In [110]:
coltons_cols = ['Genomic Description (GRCh37)', 'HGVS Nucleotide Change', 'HGVS Protein Change', 'Phenotype', 'Consequence', 'Exon', 'Intron', 'Protein Position', 'Impact', 'Interpro Domain', 'Max MAF Source', '|ddG|']

In [128]:
working_df.loc[:, ['VEP_CADD_RAW', 'VEP_CADD_PHRED', 'Max MAF (%)','Surface Area','Normalized SA','Confidence Score', 'ddG', '|ddG|']]

,VEP_CADD_RAW,VEP_CADD_PHRED,Max MAF (%),Surface Area,Normalized SA,Confidence Score,ddG,|ddG|
Genomic Description (GRCh37),,,,,,,,
20:21686358:T>C,3.009015,23.60,0.000000,172.951510,0.752780,32.73,-1.4,1.4
20:21686361:C>T,2.751772,23.00,0.006660,139.065932,0.883870,38.06,-0.0,0.0
20:21686379:G>A,2.662314,22.90,0.010493,180.703642,0.670700,37.23,-0.3,0.3
20:21686382:C>T,2.257539,21.80,3.187450,76.425233,0.601022,34.61,-0.3,0.3
20:21686390:G>T,2.319001,22.10,0.000000,99.401913,0.578281,31.28,-0.6,0.6
...,...,...,...,...,...,...,...,...
10:8115915:C>T,3.938849,27.20,0.000000,136.946574,0.870607,47.29,-0.3,0.3
10:8115916:C>G,3.979789,27.50,0.006484,136.946574,0.870607,47.29,-0.3,0.3
10:8115916:C>T,4.163516,29.20,0.012309,136.946574,0.870607,47.29,-0.2,0.2
